<a href="https://colab.research.google.com/github/Ishrak-DataScience/EmotionDetection/blob/main/Draft2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
import pandas as pd

# Load the eng.csv file
file_path = '/content/eng.csv'  # Replace 'your_folder_name' with the actual folder path

# Load the CSV file into a DataFrame
eng_df = pd.read_csv(file_path)

In [16]:
# Display the first 5 rows
eng_df.head()

,id,text,Anger,Fear,Joy,Sadness,Surprise
0,eng_train_track_a_00001,But not very happy.,0,0,1,1,0
1,eng_train_track_a_00002,Well she's not gon na last the whole song like...,0,0,1,0,0
2,eng_train_track_a_00003,She sat at her Papa's recliner sofa only to mo...,0,0,0,0,0
3,eng_train_track_a_00004,"Yes, the Oklahoma city bombing.",1,1,0,1,1
4,eng_train_track_a_00005,They were dancing to Bolero.,0,0,1,0,0


In [18]:
# Check for missing values
eng_df.isnull().sum()

,0
id,0
text,0
Anger,0
Fear,0
Joy,0
Sadness,0
Surprise,0


In [20]:
# Display data types and basic statistics
eng_df.info()        # Data types and column info
eng_df.describe()    # Summary statistics for numerical columns

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2768 entries, 0 to 2767
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        2768 non-null   object
 1   text      2768 non-null   object
 2   Anger     2768 non-null   int64 
 3   Fear      2768 non-null   int64 
 4   Joy       2768 non-null   int64 
 5   Sadness   2768 non-null   int64 
 6   Surprise  2768 non-null   int64 
dtypes: int64(5), object(2)
memory usage: 151.5+ KB


,Anger,Fear,Joy,Sadness,Surprise
count,2768.000000,2768.000000,2768.000000,2768.000000,2768.000000
mean,0.120303,0.582009,0.243497,0.317197,0.303107
std,0.325375,0.493318,0.429270,0.465469,0.459684
min,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,1.000000,0.000000,0.000000,0.000000
75%,0.000000,1.000000,0.000000,1.000000,1.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000


In [21]:
# Display column names
print("Column names:", eng_df.columns.tolist())

Column names: ['id', 'text', 'Anger', 'Fear', 'Joy', 'Sadness', 'Surprise']


In [22]:
# Verify that 'Disgust' column exists; if not, add it with default 0 values
if 'Disgust' not in eng_df.columns:
    eng_df['Disgust'] = 0  # Add 'Disgust' column with no disgust if it's not included

# Define required columns
required_columns = ['Anger', 'Fear', 'Joy', 'Sadness', 'Surprise', 'Disgust']
print("Available columns:", eng_df.columns)
print("Missing columns:", [col for col in required_columns if col not in eng_df.columns])

Available columns: Index(['id', 'text', 'Anger', 'Fear', 'Joy', 'Sadness', 'Surprise', 'Disgust'], dtype='object')
Missing columns: []


In [23]:
from sklearn.model_selection import train_test_split

# Features and target labels
X = eng_df['text']          # Text data
y = eng_df[required_columns] # Emotion labels

# Split into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [24]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Use TF-IDF vectorizer
vectorizer = TfidfVectorizer(max_features=5000)
X_train_tfidf = vectorizer.fit_transform(X_train)
X_val_tfidf = vectorizer.transform(X_val)

In [25]:
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import classification_report

# Initialize the model with OneVsRestClassifier
model = OneVsRestClassifier(LogisticRegression())

# Train the model
model.fit(X_train_tfidf, y_train)

# Predict on validation data
y_pred = model.predict(X_val_tfidf)

# Display evaluation metrics
print("Classification Report:\n", classification_report(y_val, y_pred, target_names=required_columns))

Classification Report:
               precision    recall  f1-score   support

       Anger       0.00      0.00      0.00        61
        Fear       0.62      0.89      0.73       314
         Joy       0.60      0.02      0.04       134
     Sadness       0.68      0.13      0.22       171
    Surprise       0.78      0.26      0.39       172
     Disgust       0.00      0.00      0.00         0

   micro avg       0.64      0.41      0.50       852
   macro avg       0.45      0.22      0.23       852
weighted avg       0.62      0.41      0.40       852
 samples avg       0.53      0.38      0.42       852



/usr/local/lib/python3.10/dist-packages/sklearn/multiclass.py:87: UserWarning: Label not 5 is present in all training examples.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to c

In [28]:
# New text snippet for prediction
sample_text = ["I am thrilled about the new project!"]

# Transform and predict
sample_text_tfidf = vectorizer.transform(sample_text)
sample_pred = model.predict(sample_text_tfidf)

# Convert prediction to a dictionary for easy interpretation
predicted_emotions = dict(zip(required_columns, sample_pred[0]))
print("Predicted Emotions for Sample Text:", predicted_emotions)

Predicted Emotions for Sample Text: {'Anger': 0, 'Fear': 1, 'Joy': 0, 'Sadness': 0, 'Surprise': 0, 'Disgust': 0}
